In [1]:
from torchDataLoader import FaceDataLoader

In [2]:
from tqdm import tqdm

In [3]:
import torch

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [5]:
device

device(type='cuda')

In [6]:
dset_train = FaceDataLoader()

In [7]:
from torch.utils.data import Dataset, DataLoader

In [8]:
train_loader = DataLoader(dset_train, batch_size=10, shuffle=True)

In [9]:
train_loader

In [10]:
from model import EmoModel
model = EmoModel()

In [11]:
model.cuda(device)
model

EmoModel(
  (conv1): Sequential(
    (0): Conv2d(3, 64, kernel_size=(5, 5), stride=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv2): Sequential(
    (0): Conv2d(64, 64, kernel_size=(5, 5), stride=(1, 1))
    (1): ReLU(inplace)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv3): Sequential(
    (0): Conv2d(64, 128, kernel_size=(4, 4), stride=(1, 1))
    (1): ReLU(inplace)
  )
  (conv4): Sequential(
    (0): Conv2d(128, 512, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU(inplace)
  )
  (conv5): Sequential(
    (0): Conv2d(512, 512, kernel_size=(3, 3), stride=(2, 2))
    (1): ReLU(inplace)
  )
  (Maxpool): MaxPool2d(kernel_size=3, stride=3, padding=0, dilation=1, ceil_mode=False)
  (fc6): Sequential(
    (0): Dropout(p=0.3)
    (1): Linear(in_features=25088, out_features=404

In [12]:
import torch.optim as optim
import torch.nn as nn
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

In [13]:

num_epochs = 200 # for demonstration purpose I have kept num epochs as 1. For full training make it 10 or 20 or even more if you want to see performance
total_step = len(train_loader)
for epoch in range(num_epochs):
    train_loader = tqdm(train_loader)
    for i, (images, labels) in enumerate(train_loader):
        # CrossEntropy takes target indices not the one hot vector.
        # values, indices = torch.max(labels)
        _, targets = torch.max(labels, -1)
        # Move tensors to the configured device
        images = images.to(device)
        targets = targets.to(device)
        # Forward pass - We will be working on complete batch of 16 images. And not with 1 image at a time
        # remember the images tensor is of shape -> [16, 3, 64, 64],
        # labels tensor is of shape -> [16, 2] and targets tensor is of shape -> [16,]
        outputs = model(images)

        loss = criterion(outputs, targets)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        """        for i in range(len(images)):
            outputs = model(images[i].unsqueeze_(0))
            loss = criterion(outputs[i], labels[i])
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()"""
        
        #if (i+1) % 100 == 0:
        train_loader.set_description('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
               .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

  0%|                                                                                                                                                                                      | 0/22 [00:00<?, ?it/s]D:\workplace\facial_recog\model.py:70: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  out = F.log_softmax(self.fc8(out))
Epoch [200/200], Step [22/22], Loss: 0.2586: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 22/22 [00:07<00:00,  3.10it/s]


In [28]:
torch.save(model, 'filename.pt')

In [29]:
torch.save(model.state_dict(), 'filename2.pt')

In [30]:
import cv2
img_data = cv2.cvtColor(cv2.imread("process/0_57.tiff"), cv2.COLOR_BGR2RGB)

In [33]:
img_data.shape
import numpy as np

In [34]:
img_data = np.rollaxis(img_data, 2, 0).astype('float32')

In [35]:
img_data.shape

(3, 224, 224)

In [41]:
img_data = np.reshape(img_data, (1,3, 224, 224))

In [42]:
img_data.shape

(1, 3, 224, 224)

In [43]:
image = torch.from_numpy(img_data)

In [46]:
image.shape

torch.Size([1, 3, 224, 224])

In [47]:
image = image.to(device)

In [48]:
pred = model(image)

D:\workplace\facial_recog\model.py:70: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  out = F.log_softmax(self.fc8(out))


In [49]:
pred

tensor([[ -0.0087,  -6.5696,  -6.4743,  -5.3997,  -6.7383, -13.2337]],
       device='cuda:0', grad_fn=<LogSoftmaxBackward>)